In [ ]:
config = {
    "pretrained_model": "bert-base-uncased",
    "tokenizer": "bert-base-uncased",
    "max_seq_length": 256,
    "batch_size": 16,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "0",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 2,
    "is_multilabel": False,
    "valid_metric": "f1",
    "model_save_dir": "../checkpoints/bert_uncased_256_biocaster_binary_20200903/",
    "patience": 4,
}

In [2]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaser2text,biocaster2df
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
data_df = biocaser2text(data_file)
data_df["source"] = "Biocaster"
data_df.head(3)

parse biocaser data from /home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels,source
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0,Biocaster
1,\nTyphoid outbreak in Agusan del Sur town unde...,1,Biocaster
2,\n Typhoid Outbreak In Central Nepal November...,1,Biocaster


In [3]:
from sklearn.utils import shuffle
data_df = shuffle(data_df).reset_index()
data_df["flag"]=None
train_index = int(len(data_df.index)*0.8)
valid_index = train_index + round(len(data_df.index)*0.1)
data_df.iloc[:train_index]["flag"]="train"
data_df.iloc[train_index:valid_index]["flag"]="valid"
data_df.iloc[valid_index:]["flag"]="test"

<ipython-input-3-79406d09754d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[:train_index]["flag"]="train"
<ipython-input-3-79406d09754d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[train_index:valid_index]["flag"]="valid"
<ipython-input-3-79406d09754d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [4]:
train_set = data_df[data_df["flag"]=="train"]
dev_set = data_df[data_df["flag"]=="valid"]
test_set = data_df[data_df["flag"]=="test"]

In [5]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [6]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [7]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [8]:
cls.train(train_set,dev_set)

Evaluating:  29%|██▊       | 2/7 [00:00<00:00, 11.10it/s]

Execute [train_an_epoch] method costing 14021.00 ms
[Epoch 0] loss: 33.19612929224968



Evaluating: 100%|██████████| 7/7 [00:00<00:00, 12.34it/s]


Execute [eval] method costing 570.57 ms
------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------+----------+
|    | metrics   |   values |
|----+-----------+----------|
|  0 | accuracy  | 0.88     |
|  1 | precision | 0.789474 |
|  2 | recall    | 0.882353 |
|  3 | f1        | 0.833333 |
+----+-----------+----------+
------------------------------------------------------------


Evaluating:  29%|██▊       | 2/7 [00:00<00:00, 10.99it/s]

...

Execute [eval] method costing 599.62 ms



Evaluating:  29%|██▊       | 2/7 [00:00<00:00, 10.70it/s]

Execute [train_an_epoch] method costing 14368.47 ms
[Epoch 6] loss: 2.3766325041651726



Epoch:  60%|██████    | 6/10 [01:44<01:09, 17.42s/it]

Execute [eval] method costing 592.68 ms
Early Stopping. Epoch: 6, best_valid_metric (f1): 0.8888888888888888


In [9]:
cls.test(test_set)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 11.91it/s]

Execute [eval] method costing 591.10 ms


{'accuracy': 0.9207920792079208,
 'precision': 0.8409090909090909,
 'recall': 0.9736842105263158,
 'f1': 0.9024390243902439}

In [10]:
test_set.iloc[1:10]

,index,docs,labels,source,flag
903,185,\nArchive Number 20061109.3216 \nPublished Dat...,1,Biocaster,test
904,698,\n India carries out huge bird cull Health of...,1,Biocaster,test
905,676,\nNYT-03-05-96 2001EST\nBC-TAIWAN-STOCKS-BLOOM...,0,Biocaster,test
906,630,\n940112\nDRS Industries to Change NAME \n01/1...,0,Biocaster,test
907,621,\n930729\nEconomy: Group of Top Executives Urg...,0,Biocaster,test
908,191,\nArchive Number 20060623.1743 \nPublished Dat...,1,Biocaster,test
909,283,\n Hot weather sends more children to hospita...,0,Biocaster,test
910,507,\n PAKISTAN: Abandoned cinema becomes rehab c...,0,Biocaster,test
911,511,\n TANZANIA: Government waives tax on maize i...,0,Biocaster,test


In [11]:
cls.predict(test_set.iloc[1:10])

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 19.49it/s]

Execute [predict] method costing 103.16 ms


array([1, 1, 0, 0, 0, 1, 1, 0, 0])